In [ ]:
%load_ext autoreload
%autoreload 2
import torch, rdkit
import sys, pathlib
from pathlib import Path
_THIS_FILE = Path(globals().get("__file__", Path.cwd())).resolve()
PROJECT_ROOT = _THIS_FILE.parent
sys.path.insert(0, str(PROJECT_ROOT))
from rdkit import Chem
from utils.utils import *
from utils.dataloader import dataset
from utils.generate import *
from pathlib import Path
device   = "cuda"
vocab = dataset.vocab
index_to_token = {idx: token for token, idx in vocab.items()}

cuda


{'[#C]': 0, '[#N]': 1, '[*H0]': 2, '[=*H0]': 3, '[=Branch]': 4, '[=C]': 5, '[=N]': 6, '[=O]': 7, '[=S]': 8, '[Branch]': 9, '[C]': 10, '[Cl]': 11, '[EOS]': 12, '[F]': 13, '[NH0+1]': 14, '[N]': 15, '[OH0-1]': 16, '[O]': 17, '[PAD]': 18, '[PH1]': 19, '[P]': 20, '[Ring1]': 21, '[Ring2]': 22, '[SOS]': 23, '[S]': 24, '[SiH0]': 25, '[pop]': 26}
27
tensor([ 2, 10, 17, 10,  2, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,
        18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,
        18, 18, 18, 18])
tensor([23,  2, 10, 17, 10,  2, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,
        18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,
        18, 18, 18, 18])
tensor([ 2, 10, 17, 10,  2, 12, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,
        18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18,
        18, 18, 18, 18])


In [2]:
def select_model(choice, latent):
    if choice == "Trans_MHA":
        from models.Trans_MHA import CVAE, PriorNet
        model    = CVAE(latent_dim=latent).cuda().eval()
        model.decoder.cuda().eval()
        prior = PriorNet(y_dim=3, latent_dim=latent).cuda().eval()

        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_dmodel256.pth")
        state_dict = torch.load(save_path)
        model.load_state_dict(state_dict)

        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_prior.pth")
        state_dict = torch.load(save_path)
        prior.load_state_dict(state_dict)

    # Trans
    elif choice == "Trans":
        from models.Trans import CVAE, PriorNet
        model    = CVAE(latent_dim=latent).cuda().eval()
        model.decoder.cuda().eval()
        prior = PriorNet(y_dim=3, latent_dim=latent).cuda().eval()

        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_dmodel256_no_mha.pth")
        state_dict = torch.load(save_path)
        model.load_state_dict(state_dict)

        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_prior_no_mha.pth")
        state_dict = torch.load(save_path)
        prior.load_state_dict(state_dict)
    # LSTM
    elif choice == "LSTM":
        from models.LSTM import CVAE, PriorNet
        model    = CVAE(latent_dim=latent).cuda().eval()
        model.decoder.cuda().eval()
        prior = PriorNet(y_dim=3, latent_dim=latent).cuda().eval()

        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_LSTM.pth")
        state_dict = torch.load(save_path)
        model.load_state_dict(state_dict)

        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_LSTM_prior.pth")
        state_dict = torch.load(save_path)
        prior.load_state_dict(state_dict)


    # LSTM + MHA
    elif choice == "LSTM_MHA":
        from models.LSTM_MHA import CVAE, PriorNet
        model    = CVAE(latent_dim=latent).cuda().eval()
        model.decoder.cuda().eval()
        prior = PriorNet(y_dim=3, latent_dim=latent).cuda().eval()

        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_LSTM_MHA.pth")
        state_dict = torch.load(save_path)
        model.load_state_dict(state_dict)

        save_path = (PROJECT_ROOT / "models/weights" / "model_weights_LSTM_MHA_prior.pth")
        state_dict = torch.load(save_path)
        prior.load_state_dict(state_dict)
    

    return model, prior

In [3]:
def select_kernel(choice, model, prior, condition):
    condition = condition.repeat(1024, 1, 1)
    # Trans + MHA
    if choice == "Trans_MHA":
        mean, var = prior(condition.squeeze())
        z = model.reparameterize(mean, var)
        condition = model.input_embedding_p(condition)
        condition = model.pos_enc(condition)
        z_prime = model.ff(model.crossattn(z, condition, condition))

    # Trans
    elif choice == "Trans":
        mean, var = prior(condition.squeeze())
        z_prime = model.reparameterize(mean, var)


    # LSTM
    elif choice == "LSTM":
        mu, lv = prior(condition.squeeze())
        z = model.reparameterize(mu, lv)
        z_prime = z

    # LSTM + MHA
    elif choice == "LSTM_MHA":
        mean, var = prior(condition.squeeze())
        z = model.reparameterize(mean, var)
        condition = model.input_embedding_p(condition)
        z_prime = model.ff(model.crossattn(z, condition, condition))
    
    return z_prime, condition

In [4]:
latent_dim = 128
#condition = torch.tensor([21.0, -0.4, 1.4]).unsqueeze(-1).to(device) 
condition = torch.tensor([0, -0.4, 1.4]).unsqueeze(-1).to(device)
mode = "LSTM_MHA"
model, prior = select_model(mode, latent_dim)
z_prime, props = select_kernel(mode, model, prior, condition)

# 예시 호출 (z_prime는 shape [1, 265, 64]라고 가정)벡터
if mode =="Trans" or mode =="Trans_MHA":
    generated_tokens = generate_batch_sequence(model, z_prime,
                                        max_length=42, start_token=vocab['[SOS]'],
                                        end_token=vocab['[EOS]'],
                                        pad_token=vocab['[PAD]'], fixed_seq_len=42,
                                        device=device)
elif mode=="LSTM": # LSTM
    generated_tokens = generate_batch_sequence_LSTM(model, z_prime, start_token=vocab['[SOS]'],
                                                    end_token=vocab['[EOS]'],
                                                    pad_token=vocab['[PAD]'],
                                                    device=device, grammar_mask_fn=None,
                                                    index_to_token=index_to_token)
else:
    generated_tokens = generate_batch_sequence_LSTM_MHA(model, z=z_prime, start_token=vocab['[SOS]'],
                                                        end_token=vocab['[EOS]'],
                                                        pad_token=vocab['[PAD]'],
                                                        grammar_mask_fn=None, index_to_token=index_to_token)
print("Generated token sequence:", generated_tokens)

Generated token sequence: [[23, 2, 10, 15, 15, 15, 10, 10, 10, 10, 10, 10, 1, 1, 26, 17, 17, 10, 4, 7, 7, 17, 17, 10, 10, 10, 10, 10, 10, 12], [23, 2, 10, 10, 10, 10, 10, 10, 10, 9, 9, 13, 13, 26, 26, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 12], [23, 2, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 12], [23, 2, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 12], [23, 2, 10, 15, 9, 9, 10, 10, 10, 10, 10, 10, 10, 13, 13, 26, 26, 15, 15, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 12], [23, 2, 10, 15, 15, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 17, 17, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 12], [23, 2, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 2, 2, 26, 7, 7, 12], [23, 2, 10, 15, 15, 15, 10, 10, 10, 10, 10, 10, 10, 26, 26, 10, 10, 10, 10, 10,

In [5]:
from rdkit import Chem, RDLogger
from rdkit.Chem import DataStructs, rdFingerprintGenerator
RDLogger.DisableLog('rdApp.error')

generated_smiles = []

canbe = 0
notbe = 0
for smi in generated_tokens:
    try:
        del smi[0]
        del smi[-1]
        smiles = tok_ids_to_smiles(smi)
        
        PS(smiles)
        print(PS(smiles).psmiles)
        canbe += 1
        generated_smiles.append(smiles)
    except:
        notbe += 1


print("가능한 분자 개수 :", canbe)
print("불가능한 분자 개수 :", notbe)


[*]CCCCCCNNNOONNNCNCCCCC[*]
[*]CNNCCCCCCCCCCCOOC(=O)OOOC([*])C
[*]CCCCCCCCCCCCNNC([*])OO
[*]CN(CCCCCC#N)N=NOOCOOC([*])CCCCC
[*]COOOOONNNN([*])CCCCCC
[*]CCCCNCNNNONNNCCCC[*]
[*]CCCCCCCCCCNNNCCCNC([*])C
[*]CNNOOSN([*])CCCCCC
[*]CNNCCCCCCCCCCCCC(O)NN([*])CC
[*]CNNCCCC(CC(F)OOOOOO)NN([*])C
[*]CN(CCC=N)OOC(O)NN([*])CC
[*]NCCCCN(NNN([*])CCCCCCC)OO
[*]CCCCCCCCCCCNNCCCOOC([*])O
[*]OOCCCCCCCCCCCOOC([*])CCCCCC
[*]C[*]
가능한 분자 개수 : 15
불가능한 분자 개수 : 1009


In [ ]:
from fcd_torch import FCD
from pathlib import Path

original_smiles = pd.read_csv(PROJECT_ROOT / "data/simulation-trajectory-aggregate_aligned.csv").iloc[:,1].values
calc       = FCD(device=device, n_jobs=8)

fcd_score  = calc(generated_smiles, original_smiles)
print(f"FCD = {fcd_score:.3f}")


FCD = 17.212
